## Pre-processing Data

In this section, we'll be focusing on preparing our data for machine learning model. We will be creating dummy variables for discreet variables, using standardized scale for numerical data, and creating our training and testing data for the model. 


In [116]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [117]:
#Loading Energy Efficiency dataframe
#Printing first five rows
df = pd.read_excel('../data/raw/energy_efficiency_file.xlsx')
df.head()


,X1,X2,X3,X4,X5,X6,X7,X8,Y1,Y2
0,0.98,514.5,294.0,110.25,7.0,2,0.0,0,15.55,21.33
1,0.98,514.5,294.0,110.25,7.0,3,0.0,0,15.55,21.33
2,0.98,514.5,294.0,110.25,7.0,4,0.0,0,15.55,21.33
3,0.98,514.5,294.0,110.25,7.0,5,0.0,0,15.55,21.33
4,0.90,563.5,318.5,122.50,7.0,2,0.0,0,20.84,28.28


In [118]:
#Renaming colummn names for clarity on what 'X' and 'Y' values represent
#Printing first five rows of dataframe

df.rename(columns= {'X1': 'Relative Compactness', 'X2': 'Surface Area', 'X3': 'Wall Area', 'X4':'Roof Area', 'X5': 'Overall Height', 'X6': 'Orientation', 'X7': 'Glazing Area', 'X8': 'Glazing Area Distribution', 'Y1': 'Heating Load', 'Y2': 'Cooling Load'}, inplace = True)
df.head()

,Relative Compactness,Surface Area,Wall Area,Roof Area,Overall Height,Orientation,Glazing Area,Glazing Area Distribution,Heating Load,Cooling Load
0,0.98,514.5,294.0,110.25,7.0,2,0.0,0,15.55,21.33
1,0.98,514.5,294.0,110.25,7.0,3,0.0,0,15.55,21.33
2,0.98,514.5,294.0,110.25,7.0,4,0.0,0,15.55,21.33
3,0.98,514.5,294.0,110.25,7.0,5,0.0,0,15.55,21.33
4,0.90,563.5,318.5,122.50,7.0,2,0.0,0,20.84,28.28


In [119]:
#Dropping the Cooling Load (y2) column, as the only Y variable we'll only be focusing on is Heating Load (y1) in this analysis. 

df.drop('Cooling Load', axis = 1, inplace = True)
df.head()

,Relative Compactness,Surface Area,Wall Area,Roof Area,Overall Height,Orientation,Glazing Area,Glazing Area Distribution,Heating Load
0,0.98,514.5,294.0,110.25,7.0,2,0.0,0,15.55
1,0.98,514.5,294.0,110.25,7.0,3,0.0,0,15.55
2,0.98,514.5,294.0,110.25,7.0,4,0.0,0,15.55
3,0.98,514.5,294.0,110.25,7.0,5,0.0,0,15.55
4,0.90,563.5,318.5,122.50,7.0,2,0.0,0,20.84


We are only interested in analyzing the relationship between building features and Heating Load in this analysis. Thus, we've decided to remove Cooling Load, the second y variable. 

In [120]:
#Checking dataframe size and data type
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Relative Compactness       768 non-null    float64
 1   Surface Area               768 non-null    float64
 2   Wall Area                  768 non-null    float64
 3   Roof Area                  768 non-null    float64
 4   Overall Height             768 non-null    float64
 5   Orientation                768 non-null    int64  
 6   Glazing Area               768 non-null    float64
 7   Glazing Area Distribution  768 non-null    int64  
 8   Heating Load               768 non-null    float64
dtypes: float64(7), int64(2)
memory usage: 54.1 KB


In [121]:
#Checking for null values
df.isna().sum()

Relative Compactness         0
Surface Area                 0
Wall Area                    0
Roof Area                    0
Overall Height               0
Orientation                  0
Glazing Area                 0
Glazing Area Distribution    0
Heating Load                 0
dtype: int64

We do not have any missing values present in our dataset. 

## Creating Dummy Variables

We'll generate dummy variables for our discrete features, namely, Orientation and Glazing Area Distribution. Despite their integer nature, both Orientation and Glazing Area act as categorical variables. We'll employ the pandas.get_dummies() function to specifically choose Orientation and Glazing Area Distribution, transforming them into dummy variables.

In [122]:
#Selecting the integer values (Orientation and Glazing Area Distribution)
df_int = df.select_dtypes(include = 'int').columns
df_int

Index(['Orientation', 'Glazing Area Distribution'], dtype='object')

In [123]:
df_dummies = pd.get_dummies(df, columns = df_int, drop_first= True)
df_dummies.head()

,Relative Compactness,Surface Area,Wall Area,Roof Area,Overall Height,Glazing Area,Heating Load,Orientation_3,Orientation_4,Orientation_5,Glazing Area Distribution_1,Glazing Area Distribution_2,Glazing Area Distribution_3,Glazing Area Distribution_4,Glazing Area Distribution_5
0,0.98,514.5,294.0,110.25,7.0,0.0,15.55,False,False,False,False,False,False,False,False
1,0.98,514.5,294.0,110.25,7.0,0.0,15.55,True,False,False,False,False,False,False,False
2,0.98,514.5,294.0,110.25,7.0,0.0,15.55,False,True,False,False,False,False,False,False
3,0.98,514.5,294.0,110.25,7.0,0.0,15.55,False,False,True,False,False,False,False,False
4,0.90,563.5,318.5,122.50,7.0,0.0,20.84,False,False,False,False,False,False,False,False


In [124]:
bool_to_int = ["Orientation_3", "Orientation_4", "Orientation_5", "Glazing Area Distribution_1", "Glazing Area Distribution_2", "Glazing Area Distribution_3", "Glazing Area Distribution_4", "Glazing Area Distribution_5"]

for column in bool_to_int:
    df_dummies[column] = df_dummies[column].astype(int)


In [125]:
df_dummies.dtypes
df_dummies.head()

,Relative Compactness,Surface Area,Wall Area,Roof Area,Overall Height,Glazing Area,Heating Load,Orientation_3,Orientation_4,Orientation_5,Glazing Area Distribution_1,Glazing Area Distribution_2,Glazing Area Distribution_3,Glazing Area Distribution_4,Glazing Area Distribution_5
0,0.98,514.5,294.0,110.25,7.0,0.0,15.55,0,0,0,0,0,0,0,0
1,0.98,514.5,294.0,110.25,7.0,0.0,15.55,1,0,0,0,0,0,0,0
2,0.98,514.5,294.0,110.25,7.0,0.0,15.55,0,1,0,0,0,0,0,0
3,0.98,514.5,294.0,110.25,7.0,0.0,15.55,0,0,1,0,0,0,0,0
4,0.90,563.5,318.5,122.50,7.0,0.0,20.84,0,0,0,0,0,0,0,0


## Scaling Data

In this section we'll be utilizing MinMaxScaler which will place our data between a range of 0-1. Our dataset is comprised of variables with different units and extreme scale differences (e.g.: The scale of Surface Area vs scale of Relative Compactness). We will split data into training and test (using a 75/25) split. We will perform fit only on the training data, mitigating the risk of data leakage and overfitting on the test set.


In [131]:
df = df_dummies

In [132]:
#X - Dropping y variable
#y - Is our target variable
X = df.drop('Heating Load', axis = 1)
y = df['Heating Load']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 33)


In [136]:
X_scaling = ["Relative Compactness", "Surface Area", "Wall Area", "Roof Area", "Overall Height"]

In [135]:
#Storing Scaler into X_scale variable
X_scale = MinMaxScaler()

#Fitting X data 
X_scale.fit(X_train[X_scaling])

#Transforming X data for training and testing
X_train_scale = X_scale.transform(X_train[X_scaling])
X_test_scale = X_scale.transform(X_test[X_scaling])


In [137]:
#Saving dataframe for modeling notebook

from sklearn import datasets
%store df

Stored 'df' (DataFrame)


## Pre-Processing Recap

ENTER TEXT HERE